# Import libraries

In [1]:
#import requited libraries 
from requests import get
from bs4 import BeautifulSoup
from xlsxwriter import Workbook
import xlrd
import pandas as pd
import re

import numpy as np
import pandas as pd
import scipy.stats
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from math import sqrt
import seaborn as sns
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#check current working directory 
import os
print(os.getcwd())

C:\Users\malejin\Documents\Python Scripts\py_data_cleaning_loader\etl_cleaning_loader


# Set SQL Server connection and read data

In [ ]:
# import pyodbc library
import pyodbc
#check for drivers
for driver in pyodbc.drivers():
    print(driver)

In [ ]:
#set connection
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-42IUD2D;'
                      'Database=Pharmacy_Price_Comparsion;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#import ogyi_product_names from dbo.ogyi_product_list_simplified
benu_daily = pd.read_sql_query('''SELECT * from dbo.benu_daily_raw_data''', conn)

# Cleaner functions

In [37]:
#Remove html tags own function. This will be used by each section later in the code.
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

#Clean string after html removal
def cleanstring(data):
    for i in range(len(data)):
        if data!="üres":
            data=data.strip().replace('\n', ' ').replace('\\r', ' ').replace(u'\xa0',' ').replace("\\xa0","").replace("  "," ").replace(",,",",").replace(", ,",",").replace(" , ",",").replace('\t', ' ').replace("amp;","").replace(" :",":").replace(" ,",",")

            while data.endswith("]']"):
                data=re.sub(r"]']+$","", str(data)).strip()
                
            while data.startswith("['"):
                data=re.sub(r"^\['+", "", str(data)).strip()
            
            while data.endswith(("'",",")):
                data=re.sub(r"'|,+$","", str(data)).strip()
                
            while data.startswith((",",":")):
                data=re.sub(r"^\'|^\,|^\:+", "", str(data)).strip() 
            

    return data   

#removal of unwanted copy parts
def removeunwantedparts(data):
    for i in range(len(data)):
        if data!="üres":
            if 'A forgalomba hozatali engedély jogosultja' in data:
                data=data.replace('A forgalomba hozatali engedély jogosultja',"").strip()
            if 'A forgalombahozatali engedély jogosultja' in data:
                data=data.replace('A forgalombahozatali engedély jogosultja',"").strip()
            if 'és gyártó' in data:
                data=data.replace('és gyártó',"").strip()
            if 'és a gyártó' in data:
                data=data.replace('és a gyártó',"").strip()
            if 'Gyártó' in data:
                data=data.replace('Gyártó',"").strip()
            if "A készítményhez kapcsolódó további kérdéseivel forduljon a forgalomba hozatali engedély jogosultjának helyi képviseletéhez" in data:
                data=data.replace("A készítményhez kapcsolódó további kérdéseivel forduljon a forgalomba hozatali engedély jogosultjának helyi képviseletéhez","")            
            if "A készítményhez kapcsolódó további kérdéseivel forduljon a forgalomba hozatali engedély jogosultjához:" in data:
                data=data.replace("A készítményhez kapcsolódó további kérdéseivel forduljon a forgalomba hozatali engedély jogosultjához:","")
            if "Ezt a gyógyszert az Európai Gazdasági Térség tagállamaiban az alábbi neveken engedélyezték:" in data:
                data=data.split("Ezt a gyógyszert az Európai Gazdasági Térség tagállamaiban az alábbi neveken engedélyezték:")[0] 
            if "Nyilvántartási száma" in data:
                data=data.split("Nyilvántartási száma")[0]
            if "Nyilvántartási számai" in data:
                data=data.split("Nyilvántartási számai")[0]
            if "A betegtájékoztató legutóbbi felülvizsgálatának dátuma" in data:
                data=data.split("A betegtájékoztató legutóbbi felülvizsgálatának dátuma")[0]
            if "[logo'" in data:
                data=data.replace("[logo'","").strip()
            if "PVC/PVDC//Al buborékcsomagolásban" in data:
                data=data.replace("PVC/PVDC//Al buborékcsomagolásban","").strip()
            if "PVC/PE/PVDC//Al buborékcsomagolásban" in data:
                data=data.replace("PVC/PE/PVDC//Al buborékcsomagolásban","").strip()
            if "A forgalomba hozatali engedély száma" in data:
                data=data.replace("A forgalomba hozatali engedély száma","").strip()
            if "Kid 0,5 mg/ml oldatos orrspray" in data:
                data=data.replace("Kid 0,5 mg/ml oldatos orrspray","").strip()   

    return data  

In [38]:
#Remove html tags of vendor list
vendor_list_string=[]
for i in range(len(vendor_list)):
    if vendor_list[i]!='[]':
        vendor_list_string.append(cleanstring(removeunwantedparts(cleanstring(striphtml(str(vendor_list[i]))))))
    else: vendor_list_string.append("üres")  
vendor_list_string  

['Sandoz Hungária Kft., Bartók Béla út 43-47., 1114 Budapest, Magyarország',
 'Sandoz Hungária Kft., Bartók Béla út 43-47., 1114 Budapest, Magyarország',
 'Sandoz Hungária Kft.,1114 Budapest, Bartók Béla út 43-47.,Magyarország',
 'Sandoz Hungária Kft.,1114 Budapest, Bartók Béla út 43-47.,Magyarország',
 'Sandoz Hungária Kft., Bartók Béla út 43-47., 1114 Budapest, Magyarország',
 'Sandoz Hungária Kft., Bartók Béla út 43-47., 1114 Budapest, Magyarország',
 'Sandoz Hungária Kft., 1114 Budapest, Bartók Béla út 43-47., Magyarország',
 'Sandoz Hungária Kft., 1114 Budapest, Bartók Béla út 43-47., Magyarország',
 'Sandoz Hungária Kft., 1114 Budapest, Bartók Béla út 43-47., Magyarország',
 'Sandoz Hungária Kft., 1114 Budapest, Bartók Béla út 43-47., Magyarország',
 'Sandoz Hungária Kft., 1114 Budapest, Bartók Béla út 43-47., Magyarország',
 'Sandoz Hungária Kft., Bartók Béla út 43-47., 1114 Budapest, Magyarország',
 'Sandoz Hungária Kft., Bartók Béla út 43-47., 1114 Budapest, Magyarország',
 'S

# Using Regex

In [52]:
#Leaflet release date
#the original compiling had to be revised based on OGYI's splitting words. I took the common part of those words.
ad_list_e1=[]

for i in range(len(searched_product_ogyi_last50)):
    mystring = str(searched_product_ogyi_last50[i]).replace('\n', ' ').replace(u'\xa0',' ').replace("\\r","").replace("  "," ").replace("\t"," ") 
    ad=re.compile(r'(.etegtájékoztató engedélyezés.*)(?![^<>]*>)') 
    ad=ad.findall(mystring)
    ad_list_e1.append(str(ad))
#ad_list_e1
len(ad_list_e1)

273

In [53]:
#Remove html tags of ad_list items
ad_list_string=[]
for i in range(len(ad_list_e1)):
    if ad_list_e1[i]!= "[]":
        ad_list_string.append(cleanstring(striphtml(str(ad_list_e1[i]))))
    else: ad_list_string.append("üres")
        
ad_list_string

['üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'betegtájékoztató engedélyezésének dátuma: 2016. március',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'betegtájékoztató engedélyezésének dátuma: 2011. február',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'betegtájékoztató engedélyezésének dátuma: 2010. december 8.',
 'betegtájékoztató engedélyezésének dátuma: 2010. december 8.',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'betegtájékoztató engedélyezésé

In [54]:
#Split out only the 'dates'
ad_list_date=[]
for i in range(len(ad_list_string)):
    if ad_list_string[i]!="üres":
        ad_list_string[i]=ad_list_string[i].replace(",",":")
        ad_list_date.append(ad_list_string[i].split(":")[1].strip())
    else: ad_list_date.append("üres")
ad_list_date

['üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 '2016. március',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 '2011. február',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 '2010. december 8.',
 '2010. december 8.',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 '2014. november',
 '2014. november',
 '2014. november',
 'üres',
 'üres',
 'üres',
 'üres',
 'üres',
 '2013. február',
 '2012. szeptember',
 '2012. szeptember',
 '2012. szeptember',
 '2012. szeptembe

# Saving data to excel file

In [55]:
#saving scraped data 
#create new workbook for scraping data
benu_workbook= Workbook("benu_search_result_all_not_sql.xlsx")
print(benu_workbook)

#add new worksheet, also handling naming of the worksheet with the searched product name
worksheet=benu_workbook.add_worksheet()

#create columns
worksheet.write(0,0,'product_name')
worksheet.write(0,1,'price')
worksheet.write(0,2,'promo_price')
worksheet.write(0,3,'original_price')
worksheet.write(0,4,'unit_price')
worksheet.write(0,5,'ep')
worksheet.write(0,6,'availability')
worksheet.write(0,7,'product_category')
worksheet.write(0,8,'active_agent')
worksheet.write(0,9,'short_description')
worksheet.write(0,10,'ogyi')
worksheet.write(0,11,'ogyi_date')
worksheet.write(0,12,'vendor_name')
worksheet.write(0,13,'vendor_address')
worksheet.write(0,14,'manufacturer_name')
worksheet.write(0,15,'manufacturer_address')
worksheet.write(0,16,'vendor_and_manufacturer_name')
worksheet.write(0,17,'vendor_and_manufacturer_address')

#add all data as row to the excel, handling also if there is promo or not
start = timer()

for i in range(len(searched_product_all)):
    #write product_name
    worksheet.write((i+1),0,searched_product_all[i].h2.a.get_text('title').strip())
    #write product_price
    worksheet.write((i+1),1,searched_product_all[i].div.div.get('content'))
    if len(searched_product_all[i].div.div.select('p')) !=0:
        #write original_price
        worksheet.write((i+1),3,searched_product_all[i].div.div.p.span.get_text())
        #write promo_price
        worksheet.write((i+1),2,searched_product_all[i].div.div.p.next_sibling.next_sibling.span.get_text())
    else: worksheet.write((i+1),3,"üres"); worksheet.write((i+1),2,"üres")


for i in range(len(soup_list_details)):
    #write unit_price
    worksheet.write((i+1),4, searched_product_unit_price_all[i])
    #write ep
    worksheet.write((i+1),5, searched_product_ep_all[i])
    #write availability
    worksheet.write((i+1),6, searched_product_availability_all[i])
    #write product category
    worksheet.write((i+1),7, searched_product_breadcrumbs_all[i])
    #write active_agent
    worksheet.write((i+1),8, searched_product_active_agent_all[i])
    #write short_description
    worksheet.write((i+1),9, searched_product_short_description_all[i])
    #write ogyi 
    worksheet.write((i+1),10, ogyi_list_string[i])
    #write ogyi_date
    worksheet.write((i+1),11, ad_list_date[i])
    #write vendor
    if vendor_list_name[i]!="üres":
        worksheet.write((i+1),12, vendor_list_name[i].title())
    else:  worksheet.write((i+1),12, vendor_list_name[i])
    #write vendor_address
    worksheet.write((i+1),13, vendor_list_address[i])
    #write manufacturer
    if manufacturer_name[i]!="üres":
        worksheet.write((i+1),14, manufacturer_name[i].title())
    else:worksheet.write((i+1),14, manufacturer_name[i])
    #write manufacturer_address
    worksheet.write((i+1),15, manufacturer_address[i])
    #write vendor_and_manufacturer_name
    if vendor_and_manufacturer_name[i]!="üres":
        worksheet.write((i+1),16, vendor_and_manufacturer_name[i].title())
    else:worksheet.write((i+1),16, vendor_and_manufacturer_name[i])
    #write vendor_and_manufacturer_address
    worksheet.write((i+1),17, vendor_and_manufacturer_address[i])
    
end = timer()        


benu_workbook.close()


# Saving data to SQL Server

In [ ]:
#drop duplicated lines
df.drop_duplicates(subset="product_name", keep="first", inplace=True)
len(df)

In [ ]:
# Create an empty list 
row_list =[] 
  
# Iterate over each row 
for index, rows in df.iterrows(): 
    # Create list for the current row 
    my_list =[rows.product_name, rows.price, rows.promo_price, rows.original_price, rows.unit_price, rows.ep, rows.availability, rows.product_category,rows.active_agent, rows.short_description, rows.ogyi, rows.ogyi_date, rows.vendor_name, rows.vendor_address, rows.manufacturer_name, rows.manufacturer_address, rows.vendor_and_manufacturer_name, rows.vendor_and_manufacturer_address, rows.date] 
      
    # append the list to the final list 
    row_list.append(my_list)

In [ ]:
#insert my data into the database
sql='INSERT INTO dbo.raw_data(product_name,price,promo_price,original_price,unit_price,ep,availability,product_category,active_agent,short_description,ogyi,ogyi_date,vendor_name,vendor_address,manufacturer_name,manufacturer_address,vendor_and_manufacturer_name,vendor_and_manufacturer_address,date) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);'

for row in row_list:
    #print(row)
    values=(row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],row[12],row[13],row[14],row[15],row[16],row[17],row[18])
    #print(values)
    cursor.execute(sql, values)
    
#cursor.execute(sql, values)   
conn.commit()   

In [ ]:
#read back the content of the database
cursor.execute('SELECT * from dbo.raw_data')

In [ ]:
#print out the data from the database
for row in cursor:
    print(row)

In [ ]:
#close cursor&connection
cursor.close()
conn.close()